In [64]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
import statsmodels.api as sm
from functools import reduce
import scipy.stats as stats
from collections import OrderedDict
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

# suppress warning

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [65]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
pd.options.display.float_format = '{:.8f}'.format

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
new_df = pd.read_csv('/content/drive/My Drive/X999/new_df.csv')

In [68]:
new_df['building_age'] = 2024 - new_df['yearbuilt']

In [69]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']
building_columns = ['numfloors', 'unitsres', 'bldgarea', 'bldgclass','building_type', 'architectural_style', "building_age",
                    'building_category', 'is_condo','is_llc']
complaints_columns = ['air_quality', 'animal_issues', 'appliances',  'building_exterior', 'doors_windows', 'electrical_issues', 'elevator_issues',
'floors_stairs','general_complaints', 'graffiti_posting', 'heat_hot_water','homeless_issues', 'noise_complaints','other_issues', 'pest_issues',
'plumbing_issues', 'police_matters', 'public_nuisance','safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints']
other_features_columns = ['avg_idr','avg_idr_change','income_change','income_change_ratio',
'median_income', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp',
 'ep_uninsur', 'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',
 'ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white']
# new_df['avg_change_abs']

In [70]:
# idr_evi.columns.to_list()
all_columns = identifier_columns + metrics_columns + building_columns + complaints_columns + other_features_columns
len(all_columns)

84

In [71]:
feature_columns = building_columns + complaints_columns + other_features_columns
len(feature_columns)

61

In [72]:
new_df = new_df[all_columns]
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,D3,1,"1931–1950, Manhattan Modern",74.00000000,elevator,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PL

In [73]:
new_df[feature_columns].head()

,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000
1,14.00000000,1861.00000000,1658649.00000000,D3,1,"1931–1950, Manhattan Modern",74.00000000,elevator,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
2,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
3,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
4,5.00000000,20.00000000,9775.00000000,C1,0,"1900–1920, Beaux-Arts",124.00000000,walk-up,0,0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,40.00000000,0.00000000,11.00000000,0.00000000,2.00000000,4.00000000,0.00000000,0.00000000,0.00000000,1.00000000,0.00000000,58.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63

In [74]:
new_df.bldgclass.unique()

array(['D9', 'D3', 'C1', 'C4', 'C7', 'D1', 'D6', 'C9', 'D7', 'D4', 'S4',
       'C6', 'S5', 'S2', 'S3', 'C5', 'D5', 'R1', 'S9', 'D8', 'R4', 'C2',
       'RR', 'C3', 'C0', 'R2', 'B9', 'A4', 'D2', 'B3', 'B1', 'B2', 'A1',
       'A9', 'S1', 'A5', 'R3', 'A7', 'S0', 'D0', 'R9', 'A2', 'C8', 'A6',
       'A3', 'A8', 'A0', 'R5'], dtype=object)

In [75]:
new_df.building_category.unique()

array(['elevator', 'walk-up', 'condo-co-op',
       'primarily_res_with_mixed_use', 'two-family', 'single-family'],
      dtype=object)

In [76]:
unique_strings1 = new_df['bldgclass'].unique()
unique_strings2 = new_df['building_category'].unique()
unique_strings3 = new_df['architectural_style'].unique()
# unique_strings4 = new_df['decade'].unique()

string_to_label1 = {string: random.randint(1, 100) for string in unique_strings1}
string_to_label2 = {string: random.randint(1, 100) for string in unique_strings2}
string_to_label3 = {string: random.randint(1, 100) for string in unique_strings3}
# string_to_label4 = {string: random.randint(1, 100) for string in unique_strings4}

new_df['bldgclass_label'] = new_df['bldgclass'].map(string_to_label1)
new_df['building_category_label'] = new_df['building_category'].map(string_to_label2)
new_df['architectural_style_label'] = new_df['architectural_style'].map(string_to_label3)
# new_df['decade_label'] = new_df['decade'].map(string_to_label)

In [77]:
# new_df.age.unique()

In [78]:
# string_to_label4 = {'1940-1949': 1940, '1950-1959':1950, '1960-1969':1960, '1900-1909':1900, '1910-1919':1910, '1980-1989':1980, '1970-1979':1970, '1930-1939':1930, '1920-1929':1920, '1990-1999':1990, '2000-2009':2000, 'Pre-1900':1890, '2010-2020':2010}
# new_df['decade_label'] = new_df['decade'].map(string_to_label4)

In [79]:
# new_df.building_age.unique()

In [80]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass_label,building_category_label,architectural_style_label
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,95,86,72
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,D3,1,"1931–1950, Manhattan Modern",74.00000000,elevator,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,46,86,72
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000

In [81]:
new_df.drop(columns=['bldgclass', 'building_category', 'architectural_style'], inplace=True)

In [82]:
new_df.rename(columns={'bldgclass_label': 'bldgclass', 'building_category_label': 'building_category', \
                       'architectural_style_label': 'architectural_style'}, inplace=True)

In [83]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,95,86,72
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,46,86,72
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,1,58.00000000,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,95,86,69
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower East Side,2023,2023-0

In [84]:
new_df.columns

Index(['Location', 'primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'latitude', 'longitude', 'community_board',
       'council_district', 'census_tract', 'bin', 'nta', 'year', 'month_year',
       'eviction_count_nta', 'eviction_count_building', 'eviction_count_unit',
       'eviction_count_zipcode', 'numfloors', 'unitsres', 'bldgarea',
       'building_type', 'building_age', 'is_condo', 'is_llc', 'air_quality',
       'animal_issues', 'appliances', 'building_exterior', 'doors_windows',
       'electrical_issues', 'elevator_issues', 'floors_stairs',
       'general_complaints', 'graffiti_posting', 'heat_hot_water',
       'homeless_issues', 'noise_complaints', 'other_issues', 'pest_issues',
       'plumbing_issues', 'police_matters', 'public_nuisance',
       'safety_concerns', 'sanitation_issues', 'walls_ceilings',
       'total_complaints', 'avg_idr', 'avg_idr_change', '

## **Groupby nta on complaint features**

In [85]:
complaints_columns

['air_quality',
 'animal_issues',
 'appliances',
 'building_exterior',
 'doors_windows',
 'electrical_issues',
 'elevator_issues',
 'floors_stairs',
 'general_complaints',
 'graffiti_posting',
 'heat_hot_water',
 'homeless_issues',
 'noise_complaints',
 'other_issues',
 'pest_issues',
 'plumbing_issues',
 'police_matters',
 'public_nuisance',
 'safety_concerns',
 'sanitation_issues',
 'walls_ceilings',
 'total_complaints']

In [86]:
nta_df = new_df.groupby('nta')[complaints_columns].sum().reset_index()

In [87]:
nta_df.head()

,nta,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,Allerton-Pelham Gardens,32.00000000,216.00000000,42.00000000,3.00000000,987.00000000,122.00000000,121.00000000,101.00000000,47.00000000,0.00000000,677.00000000,36.00000000,10780.00000000,20.00000000,46.00000000,418.00000000,57.00000000,37.00000000,73.00000000,344.00000000,227.00000000,14386.00000000
1,Annadale-Huguenot-Prince's Bay-Eltingville,0.00000000,1.00000000,3.00000000,3.00000000,27.00000000,10.00000000,0.00000000,12.00000000,9.00000000,0.00000000,25.00000000,0.00000000,43.00000000,4.00000000,3.00000000,81.00000000,2.00000000,0.00000000,0.00000000,31.00000000,21.00000000,275.00000000
2,Arden Heights,0.00000000,2.00000000,0.00000000,0.00000000,6.00000000,8.00000000,0.00000000,2.00000000,8.00000000,0.00000000,18.00000000,0.00000000,72.00000000,1.00000000,8.00000000,38.00000000,1.00000000,0.00000000,0.00000000,32.00000000,8.00000000,204.00000000
3,Astoria,87.00000000,249.00000000,167.00000000,30.00000000,720.00000000,310.00000000,374.00000000,306.00000000,404.00000000,13.00000000,22315.00000000,69.00000000,18609.00000000,3.00000000,641.00000000,1491.00000000,232.00000000,76.00000000,149.00000000,5874.00000000,765.00000000,52884.00000000
4,Auburndale,36.00000000,6.00000000,25.00000000,0.00000000,42.00000000,8.00000000,39.00000000,39.00000000,20.00000000,0.00000000,243.00000000,0.00000000,553.00000000,0.00000000,9.00000000,121.00000000,2.00000000,0.00000000,10.00000000,157.00000000,42.00000000,1352.00000000


In [88]:
nta_df = new_df.merge(nta_df, on='nta', how='left', suffixes=('', '_nta'))

In [89]:
nta_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style,air_quality_nta,animal_issues_nta,appliances_nta,building_exterior_nta,doors_windows_nta,electrical_issues_nta,elevator_issues_nta,floors_stairs_nta,general_complaints_nta,graffiti_posting_nta,heat_hot_water_nta,homeless_issues_nta,noise_complaints_nta,other_issues_nta,pest_issues_nta,plumbing_issues_nta,police_matters_nta,public_nuisance_nta,safety_concerns_nta,sanitation_issues_nta,walls_ceilings_nta,total_complaints_nta
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,95,86,72,272.00000000,2101.00000000,252.00000000,2.00000000,1635.00000000,450.00000000,2943.00000000,495.00000000,894.00000000,18.00000000,5708.00000000,815.00000000,149133.00000000,136.00000000,2772.00000000,4075.00000000,1259.00000000,792.00000000,258.00000000,4689.00000000,880.00000000,179579.00000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,46,86,72,272.00000000,2101.00000000,252.00000000,2.00000000,1635.00000000,450.00000000,2943.00000000,495.00000000,894.00000000,18.00000000,5708.00000000,815.00000000,149133.00000000,136.00000000,2772.00000000,4075.00000000,1259.00000000,792.00000000,258.00000000,4689.00000000,880.00000000,179579.00000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-

In [90]:
nta_df = nta_df.drop(columns= feature_columns)
# remeber to drop

In [91]:
nta_df.rename(columns={'air_quality_nta': 'air_quality', 'animal_issues_nta': 'animal_issues', 'appliances_nta': 'appliances', \
                           'building_exterior_nta': 'building_exterior', 'doors_windows_nta': 'doors_windows', 'electrical_issues_nta': 'electrical_issues', \
                           'elevator_issues_nta': 'elevator_issues', 'floors_stairs_nta': 'floors_stairs', 'general_complaints_nta': 'general_complaints', \
                           'graffiti_posting_nta': 'graffiti_posting', 'heat_hot_water_nta': 'heat_hot_water', 'homeless_issues_nta': 'homeless_issues', \
                           'noise_complaints_nta': 'noise_complaints', 'other_issues_nta': 'other_issues', 'pest_issues_nta': 'pest_issues', \
                           'plumbing_issues_nta': 'plumbing_issues', 'police_matters_nta': 'police_matters', 'public_nuisance_nta': 'public_nuisance', \
                           'safety_concerns_nta': 'safety_concerns', 'sanitation_issues_nta': 'sanitation_issues', 'walls_ceilings_nta': 'walls_ceilings', \
                           'total_complaints_nta': 'total_complaints'}, inplace=True)

In [92]:
nta_df[complaints_columns] = nta_df[complaints_columns].div(5, axis=0)

In [93]:
nta_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,54.40000000,420.20000000,50.40000000,0.40000000,327.00000000,90.00000000,588.60000000,99.00000000,178.80000000,3.60000000,1141.60000000,163.00000000,29826.60000000,27.20000000,554.40000000,815.00000000,251.80000000,158.40000000,51.60000000,937.80000000,176.00000000,35915.80000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,54.40000000,420.20000000,50.40000000,0.40000000,327.00000000,90.00000000,588.60000000,99.00000000,178.80000000,3.60000000,1141.60000000,163.00000000,29826.60000000,27.20000000,554.40000000,815.00000000,251.80000000,158.40000000,51.60000000,937.80000000,176.00000000,35915.80000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,54.40000000,420.20000000,50.40000000,0.40000000,327.00000000,90.00000000,588.60000000,99.00000000,178.80000000,3.60000000,1141.60000000,163.00000000,29826.60000000,27.20000000,554.40000000,815.00000000,251.80000000,158.40000000,51.60000000,937.80000000,176.00000000,35915.80000000
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower East Side,2023,2023-06,56.72749658,0.80000000,0.00215633,67.00000000,54.40000000,420.20000000,50.40000000,0.40000000,327.00000000,90.00000000,588.60000000,99.00000000,178.80000000,3.60000000,1141.60000000,163.00000000,29826.60000000,27.20000000,554.40000000,815.00000000,251.80000000,158.40000000,51.60000000,937.80000000,176.00000000,35915.80000000
4,Lower East Side,0308915/24_212303,1004350013,0308915/24,212303,98 ST MARKS PLACE,5B,2024-10-22,MANHATTAN,10009,40.72739900,-73.98485400,3.00000000,2.00000000,32.00000000,1005824,East Village,2024,2024-10,56.72749658,0.20000000,0.01000000,67.00000000,3.20000000,19.20000000,20.00000000,3.20000000,116.00000000,34.40000000,21.20000000,34.40000000,76.20000000,6.60000000,519.60000000,28.20000000,889.00000000,1.60000000,63.40000000,216.20000000,5.00000000,2.00000000,16.80000000,180.20000000,77.60000000,2334.00000000


## **Groupby building features at nta level**

In [94]:
building_columns

['numfloors',
 'unitsres',
 'bldgarea',
 'bldgclass',
 'building_type',
 'architectural_style',
 'building_age',
 'building_category',
 'is_condo',
 'is_llc']

In [95]:
new_df.head(1)

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,95,86,72


In [96]:
# lambda x: x.mode().iloc[0]

In [97]:
nta_df_building = new_df.groupby('nta').agg({'building_age': "mean",
                                                     'numfloors': 'mean',
                                                     "unitsres": "mean",
                                                     "bldgarea":"mean",
                                                     'bldgclass': lambda x: x.mode().iloc[0],
                                                     'building_type': lambda x: x.mode().iloc[0],
                                                     'architectural_style': lambda x: x.mode().iloc[0],
                                                     'building_category': lambda x: x.mode().iloc[0],
                                                     'is_condo': lambda x: x.mode().iloc[0],
                                                     'is_llc': lambda x: x.mode().iloc[0],
                                                     }).reset_index()

In [98]:
nta_df_building.head()

,nta,building_age,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_category,is_condo,is_llc
0,Allerton-Pelham Gardens,74.46456693,2.77952756,87.48031496,87925.92913386,68,1,72,41,0,0
1,Annadale-Huguenot-Prince's Bay-Eltingville,32.23529412,2.11764706,1.94117647,2576.64705882,30,1,90,41,0,0
2,Arden Heights,46.00000000,2.17647059,1.70588235,1865.17647059,85,1,90,41,0,0
3,Astoria,86.97289157,4.35843373,85.04518072,69913.81325301,7,0,26,44,0,0
4,Auburndale,73.05555556,2.11111111,15.22222222,14308.83333333,68,1,69,41,0,0


In [99]:
nta_df = new_df.merge(nta_df_building, on='nta', how='left', suffixes=('', '_building'))

In [100]:
nta_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style,building_age_building,numfloors_building,unitsres_building,bldgarea_building,bldgclass_building,building_type_building,architectural_style_building,building_category_building,is_condo_building,is_llc_building
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,95,86,72,76.78067885,12.76240209,612.05744125,629180.43080940,46,1,69,86,0,0
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,46,86,72,76.78067885,12.76240209,612.05744125,629180.43080940,46,1,69,86,0,0
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,1,58.00000000,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.

In [101]:
nta_df = nta_df.drop(columns= building_columns)

In [102]:
nta_df.rename(columns={'building_age_building': 'building_age', 'numfloors_building': 'numfloors', 'unitsres_building': 'unitsres', 'bldgarea_building': 'bldgarea', "bldgclass_building": 'bldgclass',
                           'building_type_building': 'building_type', 'architectural_style_building': 'architectural_style', 'building_category_building': 'building_category', \
                           'is_condo_building': 'is_condo', 'is_llc_building': 'is_llc'}, inplace=True)

In [103]:
nta_df.head(1)

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,building_age,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_category,is_condo,is_llc
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,51503.63256500,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,76.78067885,12.76240209,612.05744125,629180.43080940,46,1,69,86,0,0


In [104]:
nta_df.columns.to_list(), nta_df.shape

(['Location',
  'primary_key',
  'bbl',
  'court_index_number',
  'docket_number',
  'eviction_address',
  'eviction_apartment_number',
  'executed_date',
  'borough',
  'zipcode',
  'latitude',
  'longitude',
  'community_board',
  'council_district',
  'census_tract',
  'bin',
  'nta',
  'year',
  'month_year',
  'eviction_count_nta',
  'eviction_count_building',
  'eviction_count_unit',
  'eviction_count_zipcode',
  'air_quality',
  'animal_issues',
  'appliances',
  'building_exterior',
  'doors_windows',
  'electrical_issues',
  'elevator_issues',
  'floors_stairs',
  'general_complaints',
  'graffiti_posting',
  'heat_hot_water',
  'homeless_issues',
  'noise_complaints',
  'other_issues',
  'pest_issues',
  'plumbing_issues',
  'police_matters',
  'public_nuisance',
  'safety_concerns',
  'sanitation_issues',
  'walls_ceilings',
  'total_complaints',
  'avg_idr',
  'avg_idr_change',
  'income_change',
  'income_change_ratio',
  'median_income',
  'rpl_theme1',
  'rpl_theme2',
  

In [105]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=nta_df
).fit()
print(final_model.summary())

                                  OLS Regression Results                                  
Dep. Variable:     np.log(eviction_count_nta + 1)   R-squared:                       0.693
Model:                                        OLS   Adj. R-squared:                  0.693
Method:                             Least Squares   F-statistic:                 1.154e+04
Date:                            Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                    19:21:30   Log-Likelihood:                -28153.
No. Observations:                           66395   AIC:                         5.633e+04
Df Residuals:                               66381   BIC:                         5.646e+04
Df Model:                                      13                                         
Covariance Type:                        nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----

In [106]:
nta_df.to_csv('/content/drive/My Drive/X999/nta_df.csv', index=False)

## **nta_df normalized**

In [107]:
scaler = StandardScaler()

In [108]:
nta_df_normalized = nta_df.copy()
nta_df_normalized = scaler.fit_transform(nta_df_normalized[complaints_columns + building_columns + other_features_columns])
nta_df_normalized = pd.DataFrame(nta_df_normalized, columns=complaints_columns + building_columns + other_features_columns)

In [109]:
nta_df_normalized.head(1)

,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230


In [110]:
df = nta_df['nta']
df.head()

,nta
0,Lower East Side
1,Lower East Side
2,Lower East Side
3,Lower East Side
4,East Village


In [111]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [112]:
nta_df_normalized['nta'] = nta_df['nta']
nta_df_normalized[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']] = nta_df[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']]
nta_df_normalized.head()

,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,nta,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode
0,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230,Lower East Side,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000
1,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,Lower East Side,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000
2,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,Lower East Side,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000
3,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.

In [113]:
# last_column = nta_df_normalized.pop('nta')
# nta_df_normalized.insert(0, 'nta', last_column)
# nta_df_normalized.head()

In [114]:
columns_to_move = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [115]:
for col in columns_to_move:
    last_column = nta_df_normalized.pop(col)
    nta_df_normalized.insert(0, col, last_column)
nta_df_normalized.head()

,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,bin,census_tract,council_district,community_board,longitude,latitude,zipcode,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,10002,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,10009,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,10009,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,67.00000000,0.00215633,0.80000000,56.72749658,2023-06,2023,L

In [116]:
last_column = nta_df_normalized.pop('nta')
nta_df_normalized.insert(0, 'nta', last_column)
nta_df_normalized.head()

,nta,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,bin,census_tract,council_district,community_board,longitude,latitude,zipcode,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,Lower East Side,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,10002,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,Lower East Side,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,10009,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,Lower East Side,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,10009,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,1.78535930,0.70501030,0.62149511,0.90716107,2.09849439,1.56578912,-0.62551031,1.22658769,-0.12118810,-0.74830448,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.35447784,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,Lower East Side,67.00000000,0.00215633,0.80000000,56.7274965

In [117]:
# last_columns = nta_df_normalized.pop('Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','nta','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
#                       'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_nta')
# nta_df_normalized.insert(1, last_columns)
# nta_df_normalized.head()

In [118]:
nta_df_normalized.to_csv('/content/drive/My Drive/X999/nta_df_normalized.csv', index=False)

In [119]:
# type(nta_df_normalized['eviction_count_nta'] = np

In [120]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=nta_df_normalized
).fit()
print(final_model.summary())

                                  OLS Regression Results                                  
Dep. Variable:     np.log(eviction_count_nta + 1)   R-squared:                       0.693
Model:                                        OLS   Adj. R-squared:                  0.693
Method:                             Least Squares   F-statistic:                 1.154e+04
Date:                            Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                    19:21:48   Log-Likelihood:                -28153.
No. Observations:                           66395   AIC:                         5.633e+04
Df Residuals:                               66381   BIC:                         5.646e+04
Df Model:                                      13                                         
Covariance Type:                        nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----